# Load the data

In [5]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go

import DataRetriever as dr

retriever = dr.DataRetriever()

CON_ATTRIBUTES = retriever.get_attributes(file_name='consuming_attributes.pkl')

con_df = retriever.get_data(file_name='All-Subsystems-hour-Year2.pkl')

all_consuming = retriever.get_attributes(file_name='consuming_attributes.pkl')
flex_consuming = ["Load_ClothesWasherPowerWithStandby", "Elec_PowerDishwasher", "Load_DryerPowerTotal"]
fixed_consuming = list(set(all_consuming) - set(flex_consuming))

data = pd.DataFrame(con_df[fixed_consuming].sum(axis=1).clip(lower=0) / 1e3)
data.columns = ['Consumed Energy']

In [6]:
train_val = data[: data.index[len(data) * 0.8]]
test = data[data.index[len(data) * 0.8]: ]

/tmp/ipykernel_128077/3376622727.py:1: FutureWarning: Indexing with a float is deprecated, and will raise an IndexError in pandas 2.0. You can manually convert to an integer key instead.
  train_val = data[: data.index[len(data) * 0.8]]
/tmp/ipykernel_128077/3376622727.py:2: FutureWarning: Indexing with a float is deprecated, and will raise an IndexError in pandas 2.0. You can manually convert to an integer key instead.
  test = data[data.index[len(data) * 0.8]: ]


# Sliding window function

In [7]:
def sliding_forecast(endog: pd.DataFrame, trend: str, order: tuple,
                     seasonal_order: tuple=None, train_length: int=0,
                     val_length: int=0, shift: int=0):

    train_start = 0
    train_end = train_start + (train_length * 24)
    val_start = train_end
    val_end = val_start + (val_length * 24)

    progress = 0

    rmse_list = list()

    while len(endog[: val_end]) < len(endog):
        model = SARIMAX(endog=endog[train_start : train_end],
                        trend=trend, order=order, seasonal_order=seasonal_order, initialization='approximate_diffuse')\
            .fit(low_memory=True, disp=False, full_output=False)

        predictions = pd.DataFrame(model.predict(start=endog.index[val_start],
                                                 end=endog.index[val_end - 1]))

        rmse_list.append(mean_squared_error(y_true=endog[val_start: val_end],
                                            y_pred=predictions,
                                            squared=False))

        train_start += shift * 24
        train_end = train_start + (train_length * 24)
        val_start = train_end
        val_end = val_start + (val_length * 24)

        progress += 1

    print(progress)

    return sum(rmse_list) / len(rmse_list)

# RMSE for SARIMA

In [8]:
sarima_params = [(4, 1, 2, 1, 1, 0, 168), (4, 1, 2, 0, 1, 1, 168),
                 (4, 1, 2, 1, 0, 0, 168), (4, 1, 2, 0, 0, 1, 168)]

sarima_rmses = dict()
key = 1

for params in sarima_params:
    result = sliding_forecast(endog=test["Consumed Energy"],
                              trend='n', order=params[0: 3], seasonal_order=params[3: ],
                              train_length=28, val_length=3, shift=5)
    sarima_rmses[key] = result
    print(f"RMSE for SARIMA{params} is: {round(result, 5)}")
    key += 1

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.8/dist-packages/statsmo

9
RMSE for SARIMA(4, 1, 2, 1, 1, 0, 168) is: 0.65191


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressiv

9
RMSE for SARIMA(4, 1, 2, 0, 1, 1, 168) is: 0.64691


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.8/dist-packages/statsmo

9
RMSE for SARIMA(4, 1, 2, 1, 0, 0, 168) is: 0.6162


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.8/dist-packages/

9
RMSE for SARIMA(4, 1, 2, 0, 0, 1, 168) is: 0.67196


In [9]:
best_rmse = min(sarima_rmses.values())
best_param_key = [key for key in sarima_rmses if sarima_rmses[key] == best_rmse]
best_params = sarima_params[best_param_key[0]]
best_params, best_rmse

(4, 1, 2, 0, 0, 1, 168)

In [ ]:
start_train = 0
end_train = start_train + 24*28
start_test = end_train
end_test = start_test + 3*24

model = SARIMAX(endog=test["Consumed Energy"][test.index[start_train: end_train]],
                trend='n',
                order=best_params[: 3],
                seasonal_order=best_params[3: ])\
    .fit(low_memory=True, disp=False, full_output=False)

predictions = pd.DataFrame(model.predict(start=test.index[start_test],
                                         end=test.index[end_test]))

In [13]:
predictions#.index

,predicted_mean
2015-12-18 00:00:00,1.156425
2015-12-18 01:00:00,1.107273
2015-12-18 02:00:00,1.073590
2015-12-18 03:00:00,1.010667
2015-12-18 04:00:00,1.168838
...,...
2015-12-20 20:00:00,1.554367
2015-12-20 21:00:00,1.545424
2015-12-20 22:00:00,1.432596
2015-12-20 23:00:00,1.263725


In [15]:

fig = go.Figure(go.Scattergl(
    x=test[start_train: end_train].index,
    y=test['Consumed Energy'][start_train: end_train],
    name="Training Data",
    line=dict(color='rgb(84, 84, 84)')
))

fig.add_trace(go.Scattergl(
    x=test[start_test: end_test].index,
    y=test['Consumed Energy'][start_test: end_test],
    name="Observed",
    line=dict(color='rgb(234,143,129)')
))

fig.add_trace(go.Scattergl(
    x=predictions.index,
    y=predictions["predicted_mean"],
    name="Predicted",
    line=dict(color='rgb(32,115,171)')
))

fig.update_yaxes(title="Fixed Consumption [kWh]")
fig.update_layout(template="plotly",
                  legend=dict(orientation="h",
                              yanchor="bottom",
                              y=1,
                              xanchor="left",
                              x=0))

fig.write_html("ARIMA_figs/best_consumption_arima_NEW.html")
fig.show()